In [1]:
from model import SplitTrainTest as split
from model import metrics
from model import SplitTrainTest as stt
from language import tf_idf

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd
import numpy as np
import interview_utility as ut
from sklearn.pipeline import Pipeline
from pipeline import t_listToOneHot
import apriori

Feature engineering: 
1. steps of recipes are extracted
2. n number of columns is added according to n number of steps, the <columnsToAdd> number of columns is added
3. most frequent words from each step column are extracted
4. ratings for recipes are added


In [2]:
r_recipes = pd.read_csv('/home/svetik/Notebooks/interview/food-com/RAW_recipes.csv')
r_recipes['steps'] = r_recipes['steps'].apply(lambda x : x.replace("'", '').
                                                 replace('[', '').
                                                 replace(']','').
                                                 split(','))
pip = Pipeline([
    ("stepsToColumns", ut.ColStrColumns(columns = ['steps'], columnsToAdd = 20)), 
    ("mostFrequentWordOneHotEncoded", tf_idf.freqToOneHot())
])
output = pip.transform(r_recipes[:50])

#join information from other files
r_recipes = r_recipes[['id']].join(output, how = 'inner')
interaction_df = pd.read_csv('/home/svetik/Notebooks/food-com/interactions_train.csv')
interaction_rating_df = interaction_df.groupby(['recipe_id']).mean()['rating']
interaction_rating_df = pd.DataFrame(interaction_rating_df)
interaction_rating_df.index.rename('id')
r_r = r_recipes.join(interaction_rating_df, on = 'id', how = 'inner')
r_r.index = r_r.id
r_r = r_r.drop(columns = ['id'])
r_r['rating'] = r_r['rating'].apply(lambda x: round(x))

Most frequent words:
[['oven', 'oven degre', 'degre', 'preheat', 'preheat oven'], ['mix', 'add', 'pan', 'ingredi', 'place'], ['drain', 'mix', 'stir', 'cover', 'add'], ['add', 'stir', 'boil', 'pepper', 'hour'], ['add', 'egg', 'sugar', 'cabbag', 'cover'], ['garlic', 'water', 'salt', 'egg', 'combin'], ['cook', 'season', 'add', 'pepper', 'tender'], ['stir', 'add', 'cover', 'chees', 'oat'], ['potato', 'bake', 'simmer', 'carrot', 'chile'], ['hour', 'butter', 'bake', 'add', 'touch'], ['boil', 'bake', 'chees', 'blend', 'cinnamon'], ['toss', 'heat', 'bake', 'cover', 'turkey'], ['bake', 'oil', 'serv', 'toast', 'desir'], ['stir', 'oven', 'salt', 'preheat', 'rice'], ['add', 'hour', 'potato', 'mixtur', 'cook'], ['serv', 'bake', 'overcook', 'turn', 'sugar'], ['heat', 'oil', 'freez', 'hour', 'drop'], ['piec', 'batter', 'garnish', 'feel comfort', 'comfort'], ['serv', 'cook', 'steak', 'stir', 'reheat serv'], ['hamburg', 'oven', 'flour', 'stir blend', 'stir'], ['avocado', 'bake', 'cook', 'use deep', 'fr

In [3]:
r_r = ut.dfWithZeroNullOnly(r_r.drop(columns = ['rating']))

3 values are deleted


In [4]:
r_r

,oven,preheat,add,mix,pan,place,mix_2step,cover,drain,stir,...,garnish,stir_18step,cook_18step,steak,stir_19step,oven_19step,flour,fryer,avocado,cook_20step
id,,,,,,,,,,,,,,,,,,,,,
137739,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31490,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59389,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70971,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75452,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42198,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67547,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
39959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Freq_items will include the combination of items appearing more often together. The consequence can be up to 20 items

In [5]:
freq_items = apriori.detect_freq_items(r_r, min_percentage=0.01, max_len = 20)
freq_items.head(7)

Processing 253 combinations | Sampling itemset size 11


,support,itemsets
0,0.142857,(oven)
1,0.142857,(preheat)
2,0.071429,(add)
3,0.035714,(mix)
4,0.107143,(pan)
5,0.107143,(place)
6,0.142857,(cover)


In [6]:
rules = apriori.get_association_rules(freq_items)


In [7]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(oven),(preheat),0.142857,0.142857,0.142857,1.0,7.0,0.122449,inf
1,(preheat),(oven),0.142857,0.142857,0.142857,1.0,7.0,0.122449,inf
2,(oil),(oven),0.035714,0.142857,0.035714,1.0,7.0,0.030612,inf
3,(batter),(oven),0.035714,0.142857,0.035714,1.0,7.0,0.030612,inf
4,(oven_19step),(oven),0.035714,0.142857,0.035714,1.0,7.0,0.030612,inf


Following I get the list of steps (described by frequent words ín steps). The sequence consist only from two steps.

In [36]:
seq_ = rules[(rules.support > 0.04) & (rules.lift > 1) & (rules.confidence > ) & (rules.iloc[:,2] > 0.01)][['antecedents', 'consequents']]
seq_steps = apriori.allSequencesToList(seq_)


(7, 2)

In [39]:
seq_steps

[['oven', 'preheat'],
 ['preheat', 'oven'],
 ['toss', 'place'],
 ['add_7step', 'add_4step'],
 ['add_4step', 'add_7step'],
 ['oven_13step', 'stir_7step'],
 ['stir_7step', 'oven_13step']]